In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

A steering vector represents the set of phase delays a plane wave experiences, evaluated at a set of array elements (antennas). The phases are specified with respect to an arbitrary origin.

For instance, suppose a plane wave is described by a wave vector k. If there are N elements in an antenna array, with element i having location given by:

$$ r_i = ( x_i, y_i, z_i ) $$

Then the steering vector is an Nx1 complex vector representing the relative phases at each antenna, and is given by:

$$ v(k) = \left[\begin{array}{@{}c@{}}
    e^{-jk.r_1} \\
    e^{-jk.r_2} \\
    \vdots \\
    e^{-jk.r_N}
    \end{array} \right] $$
    
The steering vector is written as v(k), to make explicit that it depends on the frequency and propagation direction (which determine k) of the plane wave.

In [3]:
def steering_vector( k0, theta, phi, d, Matxy, rad=1 ): # llc
    if rad == 1:
        deg2rad = 1
    else:
        deg2rad = np.pi / 180.0
    xsource = d * np.sin( deg2rad * theta) * np.cos( deg2rad * phi )
    ysource = d * np.sin( deg2rad * theta) * np.sin( deg2rad * phi )
    zsource = d * np.cos( deg2rad * theta)
    Mat_dxy = np.abs( Matxy - ( xsource + 1j * ysource ) )
    Mat_d   = np.abs( Mat_dxy + 1j * zsource )
    Mat_theta = np.arctan( Mat_dxy / zsource )
    A = np.exp( - 1j * k0 * Mat_d ) / Mat_d
    return A

In [4]:
def steervec( k0, theta, phi, d, column_x, row_y, rad=1 ): # ple
    if rad == 1:
        deg2rad = 1
    else:
        deg2rad = np.pi / 180.0
    xsource = d * np.sin( deg2rad * theta) * np.cos( deg2rad * phi )
    ysource = d * np.sin( deg2rad * theta) * np.sin( deg2rad * phi )
    zsource = d * np.cos( deg2rad * theta)
    Mat_dxy = ( (column_x-xsource)**2 + (row_y-ysource)**2 )**0.5
    Mat_d = ( Mat_dxy**2 + zsource**2)**0.5
    Mat_theta = np.arctan( Mat_dxy / zsource )
    A = np.exp( - 1j * k0 * Mat_d ) / Mat_d
    return A

In [5]:
def steering_vector_planar( k0, x, y, z, Matxy ): # llc
    xsource = x
    ysource = y
    zsource = z
    Mat_dxy = np.abs( Matxy - ( xsource + 1j * ysource ) )
    Mat_d   = np.abs( Mat_dxy + 1j * zsource )
    Mat_theta = np.arctan( Mat_dxy / zsource )
    A = np.exp( - 1j * k0 * Mat_d ) / Mat_d
    return A

In [6]:
deg2rad = np.pi / 180
f = 50 # frequency [GHz]
k0 = 2 * np.pi * f / 300.0

# Plate

In [7]:
# plate dimensions
Lx = 300
Ly = 300
dx = 0.5 * 300 / f # c / ( 2 * f )
dy = 0.5 * 300 / f # c / ( 2 * f )

nbx = int( np.ceil( Lx / dx ) ) + 1
nby = int( np.ceil( Ly / dy ) ) + 1
tabx = np.linspace( -0.5 * Lx, 0.5 * Lx, nbx ).reshape(-1,1)
taby = np.linspace( -0.5 * Ly, 0.5 * Ly, nby ).reshape(1,-1)
Matxy = tabx + 1j * taby

# Source

## Parameters

In [8]:
# source location
d_source = 480000
theta_source = -10.0
phi_source = 0.0
offset_source_x = 0.00
# compute source position
xsource = d_source * np.sin( deg2rad * theta_source) * np.cos( deg2rad * phi_source ) + offset_source_x
ysource = d_source * np.sin( deg2rad * theta_source) * np.sin( deg2rad * phi_source )
zsource = d_source * np.cos( deg2rad * theta_source)
source = (xsource, ysource, zsource)

# source aperture : cos(theta).^n_source
n_source = 0

## Steering vector

In [15]:
rad = 0
Mat_dxy = ( (tabx-xsource)**2 + (taby-ysource)**2 )**0.5
Mat_d = ( Mat_dxy**2 + zsource**2)**0.5
Mat_theta = np.arctan( Mat_dxy / zsource )
steervec_aperture = steervec( k0, theta_source, phi_source, d_source, tabx, taby, rad )
#steervec_aperture = steering_vector( k0, theta_source, phi_source, d_source, Matxy, rad )
A_aperture = ( np.cos( Mat_theta ) ** n_source ) * steervec_aperture_ple

# Observation

In [16]:
theta_obs = np.linspace( -30, 30, 3001 )
phi_obs = 0.0
d_obs = d_source
         
E = np.zeros( theta_obs.size, dtype=complex )

for i, theta in enumerate( theta_obs ):
    A = steervec( k0, theta, phi_obs, d_obs, tabx, taby, rad=0 )
    E[i] = np.sum( A_aperture * A )

In [17]:
fig, ax = plt.subplots(1,1)
ax.plot( theta_obs, 20 * np.log10( np.abs( E ) ) )
ax.grid()
ax.set_xlabel("theta_obs [°]")
ax.set_ylabel("observation[dB]")

Text(0, 0.5, 'observation[dB]')